# Lab III - Advanced Topics - Machine Learning II

Estudiantes: 
- Edward Alejandro Giraldo Gallón
- Mafredy Acevedo Holguín
- Laura Cristina Torres López

### 1)
In your own words, describe what vector embeddings are and what they are useful for.

Son representaciones de palabras o sentencias mediante arreglos numéricos de menor dimensionalidad, que guardan su sentido y su semántica. Estos vectores son empleados por algoritmos para el procesamiento del lenguaje natural, mediante la comparación de similitud entre vectores, por medio de criterios de distancia de acuerdo a la aplicación. Esta aproximación resulta más adecuada, debido a que la comparación directa entre palabras o sentencias resulta ser de alta complejidad y costo computacional.

### 2)
What do you think is the best distance criterion to estimate how far two embeddings (vectors) are from each other? Why?

De acuerdo con la literatura investigada, se encuentran dos tipos grandes categorias de evaluadores para estimar que tan lejos está un vector embebido de otro:  

* Evaluadores intrínsecos:

    Buscan medir la calidad propia del embebimiento de las palabras o sentencias a través de tareas propias del procesamiento de estas, como similiridad entre palabras, analogías y clasificación. Estas mediciones se hacen directamente sobre los vectores sin haberlos usado en tareas del mundo real. Algunas de las métricas más usadas son la similaridad coseno, la correlación de Spearman y la precisión, siendo la similaridad coseno la más usada debido a múltiples razones, tales como su independencia a la relevancia de las palabras o sentencias, su robustez respecto a la escala y su bajo costo computacional. Este método mide la similitud entre vectores a través del coseno del ángulo entre ellos. El problema de este tipo de métodos recae en que pueden confundir similitud con relacionamiento, por lo que no llegan a ser apropiados para todo tipo de evaluaciones.

* Evaluadores extrínsecos:

    Buscan medir el desempeño de los vectores embebidos usados como caracterísicas de entrada para tareas del mundo real (procesamiento de texto, traducciones, etc...). Con esto se logra conocer como las representaciones numéricas obtenidas afecta el rendimiento de tareas específicas paras las cuales son usados y entender la utilidad de los vectores de acuerdo al contexto. Algunos de las métricas empleadas son F1 Score y Perplexity.


 En resumen, no existe una métrica universal que sea mejor que las demás. Todo dependerá de que se pretende evaluar: la calidad del embebemiento en sí o su utilidad en alguna aplicación específica.

### 3)
Let us build a Q&A (question answering) system! 😀For this, consider the following steps:

In [ ]:
#Librerias a usar:

#Estructuras de datos:
import numpy as np

#Para trabajar con NLP (Natural Lenguage Processing)

##Natural Language Toolkit
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize#, word_tokenize

#Hugging Face
from transformers import RobertaTokenizer, RobertaModel
import torch

#Criterios de distancia entre vectores embebidos
from sklearn.metrics.pairwise import cosine_similarity


a) Pick whatever text you like, in the order of 20+ paragraphs.

In [137]:
#Se escoge un texto acerca de la segunda guerra mundial
with open('WW2.txt', 'r', encoding='utf-8') as archivo:
    # Lee todo el contenido del archivo
    texto = archivo.read()

b) Split that text into meaningful chunks/pieces.

In [138]:
sentencias=sent_tokenize(texto)
print(sentencias)

['World War II, also known as the Second World War, was a global conflict that lasted from 1939 to 1945.', 'It was triggered by the invasion of Poland by Nazi Germany in 1939, which led to the United Kingdom and France declaring war on Germany.', 'This marked the beginning of the war in Europe.', 'The war expanded as Germany invaded the Soviet Union in 1941, creating a second front in the East.', "Meanwhile, Japan's attack on Pearl Harbor in December 1941 brought the United States into the conflict, leading to a Pacific theater of war.", 'The Axis powers, including Germany, Italy, and Japan, fought against the Allied powers, which included the United States, Great Britain, the Soviet Union, and others.', 'The war involved over 30 countries and resulted in widespread devastation.', 'The Holocaust, in which millions of Jews and others were systematically murdered by the Nazis, remains one of the most horrifying aspects of World War II.', 'Concentration camps and extermination camps were 

c) Implement the embedding generation logic. Which tools and approaches would help you generate them easily and high-level?

In [33]:
# Cargar el modelo RoBERTa preentrenado y el tokenizador
model_name = 'roberta-base'  # Se puede elegir otro modelo si se desea. Para este caso se implementó el modelo Base de RoBERTa, 
#que es un modelo pre-entrenado que usa MLM (Masked Language Modeling) 
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

c:\Users\Usuario\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [140]:
# Embeber las sentencias
embeddings = []

for sentence in sentencias[:10]:
    # Tokenizar y convertir cada sentencia a tensores
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    
    # Obtener los embeddings
    with torch.no_grad():  # Para evitar que se realice el cálculo de gradientes
        outputs = model(**inputs)
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).numpy()
    
    # Agregar el embedding a la lista
    embeddings.append(sentence_embedding)


In [149]:
# Tokenizar y convertir la pregunta 1 a tensor
questions=['What was the World War II?','When did the war end?','What was The Holocaust?']
question_token = tokenizer(questions, return_tensors="pt", padding=True, truncation=True)

# Obtener los embeddings de la pregunta 1
with torch.no_grad():  # Para evitar que se realice el cálculo de gradientes
    outputs = model(**question_token)
    question_emb = outputs.last_hidden_state.mean(dim=1).numpy()

### d)
For every question asked by the user, return a sorted list of the N chunks/pieces in your text that
relate the most to the question. Do results make sense?

In [152]:
cosine_scores=[]
for i,Q in enumerate(questions):
    cosine_scores.append([])
    for sentence_emb in embeddings:
        cosine_scores[i].append(cosine_similarity(sentence_emb,question_emb[i].reshape(1, -1))[0][0])
    pos=np.argsort(np.array(cosine_scores[i]))[::-1]
    print(f'Pregunta {i+1}): {Q}')
    for i,p in enumerate(pos[:5]):
        print(f'La respuesta #{i+1} es:\n \t {sentencias[p]}')
    print('\n')


Pregunta 1): What was the World War II?
La respuesta #1 es:
 	 World War II, also known as the Second World War, was a global conflict that lasted from 1939 to 1945.
La respuesta #2 es:
 	 This marked the beginning of the war in Europe.
La respuesta #3 es:
 	 The war expanded as Germany invaded the Soviet Union in 1941, creating a second front in the East.
La respuesta #4 es:
 	 Meanwhile, Japan's attack on Pearl Harbor in December 1941 brought the United States into the conflict, leading to a Pacific theater of war.
La respuesta #5 es:
 	 The Battle of Stalingrad in 1942-1943 marked a turning point on the Eastern Front.


Pregunta 2): When did the war end?
La respuesta #1 es:
 	 This marked the beginning of the war in Europe.
La respuesta #2 es:
 	 The war involved over 30 countries and resulted in widespread devastation.
La respuesta #3 es:
 	 World War II, also known as the Second World War, was a global conflict that lasted from 1939 to 1945.
La respuesta #4 es:
 	 The Battle of St

Las respuestas obtenidas tienen sentido, siempre y cuando la pregunta realizada esté contenida en cierta proporción dentro de la respuesta, como lo son los casos de las preguntas 1) y 3). Para la pregunta 2), en la cual se utilizaron sinónimos y palabras que no están precisamente en la respuesta esperada, se obtuvieron respuestas sin alguna relación con la consulta.

## 4)
What do you think that could make these types of systems more robust in terms of semantics and functionality?

A continuación, se listan algunos aspectos que pueden ayudar a robustecer la funcionalidad y la semántica de este tipo de sistemas:

- Una de las formas de hacer el sistema más robusto puede ser agregar diccionarios que contegan sinónimos, antónimos, parafraseo de algunas oraciones comunes, con el fin de enriquecer la base de datos de tal forma que el algoritmo pueda entender mejor la semántica de las sentencias que embebe.

- Agregar funcionalidades que permitan entender errores gramáticales.

- Verificar previamente antes de la implementación de los modelos, la calidad del embebimiento mediante evaluaciones extrínsecas del mismo.

- Otra manera, es agregar formas de que el modelo aprenda y se realimente de las evaluaciones extrínsecas que se realicen sobre él.

- Realimentar el modelo constantemente, a medida de que se cuenta con más data.

- Implementar una búsqueda de respuesta basada en reglas en el cual el sistema funcione en una lógica de palabras claves y entidades; asignándo una prioridad o jerarquía a las reglas, incorporando reglas enfocadas a contextos y reglas de respaldo que indiquen un error.

- Combinar diferentes métricas que se acoplen bien a la naturaleza de los datos, de manera que sus diferentes características se complementen entre sí.

- Algoritmos capaces de generar por si mismos nuevas ideas (sentencias, paráfraseo, etc), que ayuden a aumentar la data de entrenamiento con la que contará el modelo.

- Existen modelos creados con el objetivo de identificar la similitud semántica entre preguntas y respuestas. Hacer uso de estos modelos y entrenarlos con el conjunto de datos específicos puede mejorar la precisión. 